In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using Printf

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [8]:
function get_aggregate_simulation_evaluation(folder)
    mol_type = "6r7m"
    assembly_successes = 0
    total_simulations = 0
    id_index = 1
    steps_from_init_to_success = Vector{Int}([])
    for file in readdir(folder)
        if split(file, ".")[end] == "jld2"
            try
                @load "$folder$file" input output
                exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
                exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
                theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])

                total_simulations += 1
                if theta < 2.0
                    assembly_successes += 1
                    push!(steps_from_init_to_success, argmin(output["Es"]))
                end
            catch e
                println("Error in $file")
            end
        end
    end
    return total_simulations, assembly_successes/total_simulations, sum(steps_from_init_to_success)/length(steps_from_init_to_success)
end

get_aggregate_simulation_evaluation (generic function with 1 method)

In [12]:
folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Moprphometric Approach")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Moprphometric Approach
Number of simulations: 198, Success ratio: 0.056, Steps to success: 5002


In [13]:
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Moprphometric Approach with Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Error in 11_rwm_wp_2_6r7m.jld2
Error in 12_rwm_wp_2_6r7m.jld2
Error in 13_rwm_wp_2_6r7m.jld2
Error in 15_rwm_wp_2_6r7m.jld2
Error in 16_rwm_wp_2_6r7m.jld2
Error in 17_rwm_wp_2_6r7m.jld2
Error in 18_rwm_wp_2_6r7m.jld2
Error in 21_rwm_wp_2_6r7m.jld2
Error in 23_rwm_wp_2_6r7m.jld2
Error in 24_rwm_wp_2_6r7m.jld2


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/JLD2.jl:175
┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/JLD2.jl:175


Error in 26_rwm_wp_2_6r7m.jld2
Error in 27_rwm_wp_2_6r7m.jld2


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/JLD2.jl:175


Error in 29_rwm_wp_2_6r7m.jld2
Error in 2_rwm_wp_2_6r7m.jld2
Error in 31_rwm_wp_2_6r7m.jld2
Error in 35_rwm_wp_2_6r7m.jld2
Error in 38_rwm_wp_2_6r7m.jld2
Error in 39_rwm_wp_2_6r7m.jld2
Error in 3_rwm_wp_2_6r7m.jld2
Error in 41_rwm_wp_2_6r7m.jld2
Error in 42_rwm_wp_2_6r7m.jld2
Error in 44_rwm_wp_2_6r7m.jld2
Error in 46_rwm_wp_2_6r7m.jld2


...

Error in 67_rwm_wp_2_6r7m.jld2
Error in 68_rwm_wp_2_6r7m.jld2
Error in 69_rwm_wp_2_6r7m.jld2
Error in 71_rwm_wp_2_6r7m.jld2
Error in 73_rwm_wp_2_6r7m.jld2
Error in 74_rwm_wp_2_6r7m.jld2
Error in 75_rwm_wp_2_6r7m.jld2
Error in 76_rwm_wp_2_6r7m.jld2
Error in 77_rwm_wp_2_6r7m.jld2
Error in 78_rwm_wp_2_6r7m.jld2


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/JLD2.jl:175


Error in 80_rwm_wp_2_6r7m.jld2
Error in 81_rwm_wp_2_6r7m.jld2
Error in 82_rwm_wp_2_6r7m.jld2


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/JLD2.jl:175


Error in 85_rwm_wp_2_6r7m.jld2
Error in 88_rwm_wp_2_6r7m.jld2
Error in 92_rwm_wp_2_6r7m.jld2
Error in 95_rwm_wp_2_6r7m.jld2
Error in 96_rwm_wp_2_6r7m.jld2
Error in 99_rwm_wp_2_6r7m.jld2
Moprphometric Approach with Persistence
Number of simulations: 148, Success ratio: 0.155, Steps to success: 566


In [11]:
folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Only Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Only Persistence
Number of simulations: 76, Success ratio: 0.039, Steps to success: 1018
